## Navie Bayes Classification

- 확률에 대한 4가지 모델

예: 주머니가 있는데, A상자를 열 수 있는 티켓, B상자를 열 수 있는 티켓이 있고 / A상자에는 - 흰색공, 파란색공이 들어가 있고, B상자에도 - 흰색공, 파란색공이 들어가 있다

1. 사전확률 : 상자가 A가 나올 확률 = probability(A) <- 이벤트의 앞단을 물어본다고해서, 사전확률(구할 수 있는 확률)
2. 조건부확률 : P(흰|A) -> A가 given으로 주어진 상황에서 흰공이 나올 확률을 구하면 됨
   - 상자가 A로 주어진 상태에서 공이 흰공이 나올 확률 <이벤트의 앞단이 주어지고, 뒷단을 물어보는 확률 (앞단은 이미 사건이 일어났기에, 고려대상이 아님), 구할 수 있는 확률
3. 결합확률 : P(A,B) => P(A,흰공) = 동시발생 이벤트, P(A) x P(흰|A) => 구할 수 있는 확률 
4. 주변확률 : P(흰) => P(A, 흰) + P(B, 흰) => 앞에 사건(A, B)가 발생하고, 뒤에 사건(흰색)이 발생하는 확률 => 구할 수 있는 확률
5. 사후확률 : P(A|흰) -> 이상태에서, 흰공이 A에서 나왔는지 B에서 나왔는지 => 이벤트가 끝났는데 이벤트의 순서를 역행했기 때문에 구할 수가 없는데, Bayes라는 사람이 증명해냄

Navie Bayes -> P(A|X),X = 흰색, 파란색
공식 : P(X|A) * P(A)  /  P(X)   *P(X|A)->우도(가능도), *P(A)->사전확률


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import binarize
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv("naive_bayes.txt", sep="\\t", engine="python")
df.head()

,id,gender,gen_cd,Height,hair,time
0,1,f,1,155,1,1
1,2,f,1,152,0,1
2,3,f,1,168,1,1
3,4,f,1,151,1,1
4,5,m,0,181,1,0


- time : 오전(1), 오후(0)
- hair : 머리길이 Long(1), short(0)
- 목적 : 현재 병원방문 고객의 정보를 가지고 남성인지? 여성인지? 확률을 구하는 것이 목적
   - 어떤 사람이 방문을 한 사건이 벌어지고 나서야, 이 사람이 남성인지 여성인지 그 확률 구하는 문제로 생각하면,
   - 사후확률을 구하는 것이다.

In [5]:
#연속 --> 범주
H_arr = np.array(df["Height"]).reshape(-1, 1)
df["H_cd"] = binarize(H_arr, threshold=165.0) #threshold 기준으로 1, 0으로 변경
df["H_cd"].value_counts()

H_cd
1    225
0    187
Name: count, dtype: int64

In [6]:
#나이브베이즈 분류기 생성 및 학습
cols = ["hair", "time", "H_cd"]
X = df.loc[:, cols]
y = df["gen_cd"]

model = GaussianNB()
model.fit(X, y)






GaussianNB()

In [7]:
test_data = [[0, 1, 1]] #"hair", "time, "height"
y_pred = model.predict(test_data)
print(y_pred)


[0]


c:\Users\Admin\anaconda3\envs\ML_test\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [8]:
y_pred = model.predict(X)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       209
           1       0.81      0.80      0.81       203

    accuracy                           0.81       412
   macro avg       0.81      0.81      0.81       412
weighted avg       0.81      0.81      0.81       412



- 장점 : 
    - ML 모델 중 가장 간단한 학습방법을 지향하고, 빠르고 정확한 모델
    - Computation Cost가 작은 기법
    - 적은량의 데이터로도 상당한 정확도를 보여줄 수 있음(Noise에 강건하다.)
    - 대량의 데이터셋에서 빠른속도를 유지(학습)
    - 연속형 데이터보다는 이산형 데이터에 높은 성능 가진다. -> *내부에서 우도를 사용하기 때문에 빈도가 꼭 필요하다
    - 멀티클래스도 가능

- 단점 :
    - 전제조건이 모든 변수들은 서로 독립시행이라고 가정(다중공선성에 취약하다.)
    - 확률이기때문에 모든 변수가 동등하게 중요하다고 가정
    - (모든 머신러닝의 문제점) : 데이터가 적으면 Overfiting에 빠질 가능성은 여전히 존재함.